In [4]:
!pip install pretty-midi
import sys
sys.path.append('../')

import pretty_midi
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import utils
from torchvision import datasets, transforms
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import random
%matplotlib inline

    100% |████████████████████████████████| 5.6MB 7.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 19.5MB/s ta 0:00:01
  Stored in directory: /tmp/.cache/pip/wheels/22/e7/6b/70eb5879f7dbcb4f44fee735a61d6298f9e082be8538b52422
Successfully built pretty-midi


In [5]:
#for msg in mid.play():
#    print(msg)

In [6]:
#for msg in mid.play():

   # print(msg.type, msg.note, msg.velocity, msg.time)

In [7]:
import os, argparse, time
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from keras.optimizers import SGD, Adam
from keras.models import model_from_json
from multiprocessing import Pool as ThreadPool

Using TensorFlow backend.


In [24]:
def get_data_generator(midi_paths, 
                       window_size=40, 
                       batch_size=32,
                       num_threads=8,
                       max_files_in_ram=170):

    if num_threads > 1:
        # load midi data
        pool = ThreadPool(num_threads)

    load_index = 0

    while True:
        load_files = midi_paths[load_index:load_index + max_files_in_ram]
        # print('length of load files: {}'.format(len(load_files)))
        load_index = (load_index + max_files_in_ram) % len(midi_paths)

        # print('loading large batch: {}'.format(max_files_in_ram))
        # print('Parsing midi files...')
        # start_time = time.time()
        if num_threads > 1:
            parsed = pool.map(parse_midi, load_files)
        else:
            parsed = map(parse_midi, load_files)
        # print('Finished in {:.2f} seconds'.format(time.time() - start_time))
        # print('parsed, now extracting data')

        data = _windows_from_monophonic_instruments(parsed, window_size)

        batch_index = 0
        while batch_index + batch_size < len(data[0]):
            # print('getting data...')
            # print('yielding small batch: {}'.format(batch_size))
            
            res = (data[0][batch_index: batch_index + batch_size], 
                   data[1][batch_index: batch_index + batch_size])

            yield res
            batch_index = batch_index + batch_size


In [25]:
OUTPUT_SIZE = 129 # 0-127 notes + 1 for rests




In [26]:
def parse_midi(path):
    midi = None
    try:
        midi = pretty_midi.PrettyMIDI(path)
        midi.remove_invalid_notes()
    except Exception as e:
        raise Exception(("%s\nerror readying midi file %s" % (e, path)))
    return midi


def log(message, verbose):
	if verbose:
		print('[*] {}'.format(message))

def create_experiment_dir(experiment_dir, verbose=False):
    
    # if the experiment directory was specified and already exists
    if experiment_dir != 'experiments/default' and \
       os.path.exists(experiment_dir):
    	# raise an error
    	raise Exception('Error: Invalid --experiment_dir, {} already exists' \
    		            .format(experiment_dir))

    # if the experiment directory was not specified, create a new numeric folder
    if experiment_dir == 'experiments/default':
    	
    	experiments = os.listdir('experiments')
    	experiments = [dir_ for dir_ in experiments \
    	               if os.path.isdir(os.path.join('experiments', dir_))]
    	
    	most_recent_exp = 0
    	for dir_ in experiments:
    		try:
    			most_recent_exp = max(int(dir_), most_recent_exp)
    		except ValueError as e:
    			# ignrore non-numeric folders in experiments/
    			pass

    	experiment_dir = os.path.join('experiments', 
    		                          str(most_recent_exp + 1).rjust(2, '0'))

    os.mkdir(experiment_dir)
    log('Created experiment directory {}'.format(experiment_dir), verbose)
    os.mkdir(os.path.join(experiment_dir, 'checkpoints'))
    log('Created checkpoint directory {}'.format(os.path.join(experiment_dir, 'checkpoints')),
    	verbose)
    os.mkdir(os.path.join(experiment_dir, 'tensorboard-logs'))
    log('Created log directory {}'.format(os.path.join(experiment_dir, 'tensorboard-logs')), 
    	verbose)

    return experiment_dir

In [27]:
def get_callbacks(experiment_dir, checkpoint_monitor='val_acc'):
    
    callbacks = []
    
    # save model checkpoints
    filepath = os.path.join(experiment_dir, 
                            'checkpoints', 
                            'checkpoint-epoch_{epoch:03d}-val_acc_{val_acc:.3f}.hdf5')

    callbacks.append(ModelCheckpoint(filepath, 
                                     monitor=checkpoint_monitor, 
                                     verbose=1, 
                                     save_best_only=False, 
                                     mode='max'))

    callbacks.append(ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.5, 
                                       patience=3, 
                                       verbose=1, 
                                       mode='auto', 
                                       epsilon=0.0001, 
                                       cooldown=0, 
                                       min_lr=0))

    callbacks.append(TensorBoard(log_dir=os.path.join(experiment_dir, 'tensorboard-logs'), 
                                histogram_freq=0, 
                                write_graph=True, 
                                write_images=False))

    return callbacks

In [28]:
def _windows_from_monophonic_instruments(midi, window_size):
    X, y = [], []
    for m in midi:
        if m is not None:
            melody_instruments = filter_monophonic(m.instruments, 1.0)
            for instrument in melody_instruments:
                if len(instrument.notes) > window_size:
                    windows = _encode_sliding_windows(instrument, window_size)
                    for w in windows:
                        X.append(w[0])
                        y.append(w[1])
    return (np.asarray(X), np.asarray(y))

# one-hot encode a sliding window of notes from a pretty midi instrument.
# This approach uses the piano roll method, where each step in the sliding
# window represents a constant unit of time (fs=4, or 1 sec / 4 = 250ms).
# This allows us to encode rests.
# expects pm_instrument to be monophonic.
def _encode_sliding_windows(pm_instrument, window_size):
    
    roll = np.copy(pm_instrument.get_piano_roll(fs=6).T)

    # trim beginning silence
    summed = np.sum(roll, axis=1)
    mask = (summed > 0).astype(float)
    roll = roll[np.argmax(mask):]
    
    # transform note velocities into 1s
    roll = (roll > 0).astype(float)
    
    # calculate the percentage of the events that are rests
    # s = np.sum(roll, axis=1)
    # num_silence = len(np.where(s == 0)[0])
    # print('{}/{} {:.2f} events are rests'.format(num_silence, len(roll), float(num_silence)/float(len(roll))))

    # append a feature: 1 to rests and 0 to notes
    rests = np.sum(roll, axis=1)
    rests = (rests != 1).astype(float)
    roll = np.insert(roll, 0, rests, axis=1)
    
    windows = []
    for i in range(0, roll.shape[0] - window_size - 1):
        windows.append((roll[i:i + window_size], roll[i + window_size + 1]))
    return windows

In [29]:
def get_percent_monophonic(pm_instrument_roll):
    mask = pm_instrument_roll.T > 0
    notes = np.sum(mask, axis=1)
    n = np.count_nonzero(notes)
    single = np.count_nonzero(notes == 1)
    if single > 0:
        return float(single) / float(n)
    elif single == 0 and n > 0:
        return 0.0
    else: # no notes of any kind
        return 0.0
    
def filter_monophonic(pm_instruments, percent_monophonic=0.99):
    return [i for i in pm_instruments if \
            get_percent_monophonic(i.get_piano_roll()) >= percent_monophonic]

In [30]:
def generate(model, seeds, window_size, length, num_to_gen, instrument_name):
    
    # generate a pretty midi file from a model using a seed
    def _gen(model, seed, window_size, length):
        
        generated = []
        # ring buffer
        buf = np.copy(seed).tolist()
        while len(generated) < length:
            arr = np.expand_dims(np.asarray(buf), 0)
            pred = model.predict(arr)
            
            # argmax sampling (NOT RECOMMENDED), or...
            # index = np.argmax(pred)
            
            # prob distrobuition sampling
            index = np.random.choice(range(0, seed.shape[1]), p=pred[0])
            pred = np.zeros(seed.shape[1])

            pred[index] = 1
            generated.append(pred)
            buf.pop(0)
            buf.append(pred)

        return generated

    midis = []
    for i in range(0, num_to_gen):
        seed = seeds[random.randint(0, len(seeds) - 1)]
        gen = _gen(model, seed, window_size, length)
        midis.append(_network_output_to_midi(gen, instrument_name))
    return midis

# create a pretty midi file with a single instrument using the one-hot encoding
# output of keras model.predict.
def _network_output_to_midi(windows, 
                           instrument_name='Acoustic Grand Piano', 
                           allow_represses=False):

    # Create a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()
    # Create an Instrument instance for a cello instrument
    instrument_program = pretty_midi.instrument_name_to_program(instrument_name)
    instrument = pretty_midi.Instrument(program=instrument_program)
    
    cur_note = None # an invalid note to start with
    cur_note_start = None
    clock = 0

    # Iterate over note names, which will be converted to note number later
    for step in windows:

        note_num = np.argmax(step) - 1
        
        # a note has changed
        if allow_represses or note_num != cur_note:
            
            # if a note has been played before and it wasn't a rest
            if cur_note is not None and cur_note >= 0:            
                # add the last note, now that we have its end time
                note = pretty_midi.Note(velocity=127, 
                                        pitch=int(cur_note), 
                                        start=cur_note_start, 
                                        end=clock)
                instrument.notes.append(note)

            # update the current note
            cur_note = note_num
            cur_note_start = clock

        # update the clock
        clock = clock + 1.0 / 4

    # Add the cello instrument to the PrettyMIDI object
    midi.instruments.append(instrument)
    return midi


In [31]:
keras.backend.clear_session()

In [32]:


optimizer = Adam
epoch = 0
num_layers = 4
model = Sequential()
for layer_index in range(num_layers):
    kwargs = dict() 
    kwargs['units'] = 64
    # if this is the first layer
    if layer_index == 0:
        kwargs['input_shape'] = (40, OUTPUT_SIZE)
        if num_layers == 1:
            kwargs['return_sequences'] = False
        else:
            kwargs['return_sequences'] = True
        model.add(LSTM(**kwargs))
    else:
        # if this is a middle layer
        if not layer_index == num_layers - 1:
            kwargs['return_sequences'] = True
            model.add(LSTM(**kwargs))
        else: # this is the last layer
            kwargs['return_sequences'] = False
            model.add(LSTM(**kwargs))
    model.add(Dropout(0.2))
    
model.add(Dense(OUTPUT_SIZE))
model.add(Activation('softmax'))


optimizer = Adam()

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizer,
              metrics=['accuracy'])


In [34]:




midi_files = [os.path.join("../input/music1", path) 
                  for path in os.listdir("../input/music1") \
                  if '.mid' in path or '.midi' in path]

print(len(midi_files))

experiment_dir = create_experiment_dir('experiment_dir4', 1)

val_split = 0.2 # use 20 percent for validation
val_split_index = int(float(len(midi_files)) * val_split)

# use generators to lazy load train/validation data, ensuring that the
# user doesn't have to load all midi files into RAM at once
train_generator = get_data_generator(midi_files[0:val_split_index])

val_generator = get_data_generator(midi_files[val_split_index:])

callbacks = get_callbacks(experiment_dir)
print('fitting model...')
# this is a somewhat magic number which is the average number of length-20 windows
# calculated from ~5K MIDI files from the Lakh MIDI Dataset.
magic_number = 827
batch_size = 60
start_time = time.time()
num_epochs = 20

model.fit_generator(train_generator,
                    steps_per_epoch=len(midi_files) * magic_number / batch_size, 
                    epochs=num_epochs,
                    validation_data=val_generator, 
                    validation_steps=len(midi_files) * 0.2 * magic_number / batch_size,
                    verbose=1, 
                    callbacks=callbacks,
                    initial_epoch=0)
log('Finished in {:.2f} seconds'.format(time.time() - start_time), 1)





5
[*] Created experiment directory experiment_dir4
[*] Created checkpoint directory experiment_dir4/checkpoints
[*] Created log directory experiment_dir4/tensorboard-logs
fitting model...


/opt/conda/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/20
69/68 [==============================] - 28s 403ms/step - loss: 3.4107 - acc: 0.1309 - val_loss: 5.8494 - val_acc: 0.0000e+00

Epoch 00001: saving model to experiment_dir4/checkpoints/checkpoint-epoch_001-val_acc_0.000.hdf5
Epoch 2/20
69/68 [==============================] - 23s 338ms/step - loss: 2.5484 - acc: 0.1513 - val_loss: 7.0003 - val_acc: 0.0000e+00

Epoch 00002: saving model to experiment_dir4/checkpoints/checkpoint-epoch_002-val_acc_0.000.hdf5
Epoch 3/20
69/68 [==============================] - 24s 346ms/step - loss: 2.4928 - acc: 0.1626 - val_loss: 7.4786 - val_acc: 0.0089

Epoch 00003: saving model to experiment_dir4/checkpoints/checkpoint-epoch_003-val_acc_0.009.hdf5
Epoch 4/20
69/68 [==============================] - 24s 343ms/step - loss: 2.1434 - acc: 0.2767 - val_loss: 7.9988 - val_acc: 0.0246

Epoch 00004: saving model to experiment_dir4/checkpoints/checkpoint-epoch_004-val_acc_0.025.hdf5

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.00050000

In [ ]:
# generate 10 tracks using random seeds
log('Loading seed files...', 1)
seed_generator = get_data_generator(midi_files, 
                                              window_size=40,
                                              batch_size=32,
                                              num_threads=1,
                                              max_files_in_ram=10)


X, y = next(seed_generator)
generated = generate(model, X, 40, 
                      100, 10, 'Acoustic Grand Piano')
if not os.path.isdir('output'):
    os.makedirs('output')

for i, midi in enumerate(generated):
    file = os.path.join('output', '{}.mid'.format(i + 1))
    midi.write(file.format(i + 1))
    log('wrote midi file to {}'.format(file), True)